# Arabidopsis transcriptome study under high light stress

In [1]:
# Import modules from pyseqrna package

from pyseqrna import pyseqrna_utils as pu
from pyseqrna import quality_check as qc
from pyseqrna import quality_trimming as qt
from pyseqrna import  aligners as al
from pyseqrna import pyseqrna_stats as ps
from pyseqrna import quantification as quants
# from pyseqrna import differential_expression as de
from pyseqrna import pyseqrna_plots as pp
from pyseqrna import multimapped_groups as mmg
import pandas as pd
import dill
from waiting import wait

In [2]:
# initialize the logger
from pyseqrna.pyseqrna_utils import PyseqrnaLogger

log = PyseqrnaLogger(mode='w', log='pp')

log.info("Analysis started")

[15:21:26]  <ipython-input-2-09f797b7f757> :: INFO : Analysis started


In [3]:
# Read input samples form input file
data = pu.read_input_file("/home/naveen/Documents/Phd_work/pySeqRNA/pySeqRNA/example/input_Sample.txt", "/home/naveen/Documents/Phd_work/pySeqRNA/pySeqRNA/example/data/" )
samples= data['samples']

[15:21:31]  pyseqrna_utils :: INFO : Reading input samples File 
[15:21:31]  pyseqrna_utils :: INFO : Input file /home/naveen/Documents/Phd_work/pySeqRNA/pySeqRNA/example/input_Sample.txt read succesfully
[15:21:31]  pyseqrna_utils :: INFO : Combination created succesfully from /home/naveen/Documents/Phd_work/pySeqRNA/pySeqRNA/example/input_Sample.txt
[15:21:31]  pyseqrna_utils :: INFO : targets dataframe for differenatial created succesfully from /home/naveen/Documents/Phd_work/pySeqRNA/pySeqRNA/example/input_Sample.txt


In [4]:
samples

{'M1A': ['M1A',
  'M1',
  '/home/naveen/Documents/Phd_work/pySeqRNA/pySeqRNA/example/data/SRR446027_1.fastq.gz'],
 'M1B': ['M1B',
  'M1',
  '/home/naveen/Documents/Phd_work/pySeqRNA/pySeqRNA/example/data/SRR446028_1.fastq.gz'],
 'A1A': ['A1A',
  'A1',
  '/home/naveen/Documents/Phd_work/pySeqRNA/pySeqRNA/example/data/SRR446029_1.fastq.gz'],
 'A1B': ['A1B',
  'A1',
  '/home/naveen/Documents/Phd_work/pySeqRNA/pySeqRNA/example/data/SRR446030_1.fastq.gz'],
 'V1A': ['V1A',
  'V1',
  '/home/naveen/Documents/Phd_work/pySeqRNA/pySeqRNA/example/data/SRR446031_1.fastq.gz'],
 'V1B': ['V1B',
  'V1',
  '/home/naveen/Documents/Phd_work/pySeqRNA/pySeqRNA/example/data/SRR446032_1.fastq.gz'],
 'M6A': ['M6A',
  'M6',
  '/home/naveen/Documents/Phd_work/pySeqRNA/pySeqRNA/example/data/SRR446033_1.fastq.gz'],
 'M6B': ['M6B',
  'M6',
  '/home/naveen/Documents/Phd_work/pySeqRNA/pySeqRNA/example/data/SRR446034_1.fastq.gz'],
 'A6A': ['A6A',
  'A6',
  '/home/naveen/Documents/Phd_work/pySeqRNA/pySeqRNA/example/dat

In [ ]:
outdir="pySeqRNA_results"

In [ ]:
import glob 

outa = glob.glob("pySeqRNA_results/star_results.1/*out.bam")



In [ ]:
outalign={}
for s in samples:
    for o in outa:
        if s in o:
            outalign[s]=[samples[s][0], samples[s][1],o]
        

In [ ]:
mmmgs = mmg.countMMG(sampleDict=samples, bamDict=outalign, gff="/home/naveen/Downloads/arabidopsis/Arabidopsis_thaliana.TAIR10.51.gff3")

In [ ]:
# initial quality check
outfastqc, jobfastqc = qc.fastqcRun(sampleDict=samples, slurm=True,cpu=30,mem=200, outDir=outdir)

for job in jobfastqc:
    wait(lambda: pu.check_status(job), waiting_for="quality to finish")
    log.info(f"Quality check completed for job {job}")

log.info("Read quality check completed succesfully")



In [ ]:
# Run trimming using trimming module
outtrim, jobtrim= qt.trim_galoreRun(sampleDict=samples,  slurm=True, cpu=40,mem=200)

for job in jobtrim:
    wait(lambda: pu.check_status(job), waiting_for="trimming to finish")
    log.info(f"Trimming completed for job {job}")

log.info("Read trimming completed succesfully")

In [ ]:
aligner = al.STAR_Aligner(genome="/home/naveen/Downloads/arabidopsis/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa", slurm=True, outDir=outdir)

In [ ]:
job = aligner.build_index()
wait(lambda: pu.check_status(job), waiting_for="alignment to finish")
log.info(f"Indexing completed for job {job}")
log.info("Genome indexing completed succesfully")

In [ ]:
aligner.check_index()

In [ ]:
outalign, jobalign = aligner.run_Alignment(target=outtrim, cpu=20, mem=200)

for job in jobalign:
    wait(lambda: pu.check_status(job), waiting_for="alignment to finish")
    log.info(f"Alignment completed for job {job}")
log.info("Read alignment completed succesfully")

In [ ]:
df= ps.align_stats(sampleDict=samples,trimDict=outtrim,bamDict=outalign)

In [ ]:
df.to_excel(outdir+"/alignment_stats.xlsx", index=False)

In [ ]:
a= quants.featureCount(bamDict=outalign, gff="/home/naveen/Downloads/arabidopsis/Arabidopsis_thaliana.TAIR10.51.gff3", outDir=outdir)

In [ ]:
counts= pd.read_csv(outdir+"/Counts_final.txt", sep="\t")

In [ ]:
targets= data['targets']
comb= data['combinations']

In [ ]:
comb = ['GL0.5-HL0.5','GL6-HL6','GL6-HL12','GL48-HL48','GL72-HL72','R14-HL72', 'R14-GL72']

In [ ]:
dd= de.runDESeq2(countDF=counts,targetFile=targets,design='sample', subset=False, combination=comb)

In [ ]:
dd= de.run_edgeR(countDF=counts,targetFile=targets, subset=False, combination=comb)

In [ ]:
import os 
dd.to_excel(os.path.join(outdir,"Raw_DEGs_all_edgeR.xlsx"), index=False)

In [ ]:
# dd= pd.read_excel(outdir+"/Raw_DEGs_all_edgeR.xlsx")
filtered_DEG = de.degFilter(degDF=dd, CompareList=comb, FDR=0.05, FOLD=2)

In [ ]:
summary= filtered_DEG['summary']
summary

In [ ]:
filtered_DEG.keys()

In [ ]:
summary.to_excel(outdir+"/DEG_summary.xlsx", index=False)

In [ ]:
wd= pd.ExcelWriter(os.path.join(outdir,"filtered_down_DEGs.xlsx"))
for key, value in filtered_DEG['filtereddown'].items():
    value.to_excel(wd,sheet_name=key)
    wd.save()
wd.close()

In [ ]:
pu.getGenes(os.path.join(outdir,"filtered_DEGs.xlsx"), combinations=comb)

In [ ]:
outdir="pySeqRNA_results"

In [ ]:
from pyseqrna import normalize_counts as nc

In [ ]:
rpkm = nc.Normalization(countFile=outdir+"/Counts_final.txt", featureFile="/home/naveen/Downloads/arabidopsis/Arabidopsis_thaliana.TAIR10.51.gff3")

In [ ]:
rpk = rpkm.RPKM()

In [ ]:
rpk[0].to_excel(outdir+"/RPKM.xlsx", index=False)

In [ ]:

rpk[1].savefig("cpm.png",bbbox_anchor='tight', dpi=300)

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as shc

In [ ]:
# a= np.log(rpk)

In [ ]:
r =rpk.corr()

In [ ]:
linked = shc.linkage(r, 'ward')


In [ ]:
R = shc.dendrogram(
                linked,
                truncate_mode='lastp',  # show only the last p merged clusters
                p=len(r.columns),  # show only the last p merged clusters
                no_plot=True,
                )

In [ ]:
temp = {R["leaves"][ii]: r.columns[ii] for ii in range(len(R["leaves"]))}
def llf(xx):
    return "{}".format(temp[xx])

In [ ]:
shc.dendrogram(
            linked,
            truncate_mode='lastp',  # show only the last p merged clusters
            p=len(r.columns),  # show only the last p merged clusters
            leaf_label_func=llf,
            orientation='left',
           
            leaf_font_size=8.,
            show_contracted=True,  # to get a distribution impression in truncated branches
            )
ax = plt.gca()


ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
for xlabel_i in ax.get_xticklabels():
    xlabel_i.set_visible(False)
    xlabel_i.set_fontsize(0.0)
for tick in ax.get_xticklines():
    tick.set_visible(False)

plt.savefig('deg.png', dpi=300, bbox_inches='tight')

In [ ]:
import pandas as pd
from pyseqrna import pyseqrna_plots as pp
from pyseqrna import gene_ontology as go

In [ ]:
rpk.to_excel(outdir+"/medianRatiocount.xlsx")

In [ ]:
result = pd.read_excel("pySeqRNA_results/Raw_DEGs_all.xlsx")

In [ ]:
outvolcano = os.path.join(outdir,"Volcano_Plots")
pu.make_directory(outvolcano)

In [ ]:
for c in comb:
        x,y =pp.plotVolcano(result,c,2)
        x.savefig(outvolcano+"/"+c+"_volcano.png", dpi=300)

In [ ]:
heatmap, ax = pp.plotHeatmap(result,comb,num=50, type='degs')

heatmap.savefig(os.path.join(outdir,"Top50_gene.png"), dpi=300)

In [ ]:
pu.getGenes(os.path.join(outdir,"filtered_DEGs.xlsx"),combinations=comb)

In [ ]:
outgo = os.path.join(outdir,"Gene_Ontology")
pu.make_directory(outgo)
gdata = go.query('athaliana')


In [ ]:
for c in comb:
    file = f"{outdir}/diff_genes/{c}.txt"
    ontology_results = go.enrichGO(gdata, file)
    ontology_results.to_csv(os.path.join(outgo, f"{c}_gene_ontology.txt"), sep="\t", index=False)

In [ ]:
import dill
dill.load_session("arabidopsis1.pyseqrna")

In [ ]:
from pyseqrna import pathway as pt
from statsmodels.stats.multitest import multipletests
import scipy.stats as stats

In [ ]:
outkegg = os.path.join(outdir,"KEGG_pathway")



In [ ]:
df, background_count = pt.kegg_list('ath')

In [ ]:
for c in comb:
    file = f"{outdir}/diff_genes/{c}.txt"
    kegg_results = pt.enrichKEGG(file, df, background_count)
    kegg_results.to_csv(os.path.join(outkegg, f"{c}_kegg.txt"), sep="\t", index=False)

In [ ]:

dill.load_session("arabidopsis.pyseqrna")